In [1]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA 
from langchain.document_loaders import TextLoader 
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI




In [2]:
OPENAI_API_KEY= (os.getenv("OPENAI_API_KEY"))

In [3]:
import json
from llama_index.readers.google import GoogleDriveReader

# Lade die Service-Account-Schlüsseldatei
with open("credentials.json") as f:
    service_account_key = json.load(f)

# Initialize the reader
reader = GoogleDriveReader(
    folder_id="1omQKaU6syLr5X0kcr1_Vnl0LKNfQzanI",
    service_account_key=service_account_key,  # Hier das geladene Dict übergeben!
)


In [4]:
new_documents = reader.load_data()

# Wenn du Duplikate vermeiden willst, kannst du einen Hash oder die Datei-ID als Schlüssel verwenden.
# Beispiel: Setzen eines Sets, um IDs zu speichern und Duplikate zu vermeiden
unique_documents = []
seen_ids = set()  # Hier speichern wir bereits gesehene IDs

for doc in new_documents:
    doc_id = getattr(doc, "doc_id", None)  # Annahme: Jedes Dokument hat eine ID
    if doc_id not in seen_ids:
        unique_documents.append(doc)
        seen_ids.add(doc_id)

# Die unique_documents-Liste enthält nun nur noch die eindeutigen Dokumente.
print(f"Anzahl eindeutiger Dokumente: {len(unique_documents)}")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong p

Failed to load file C:\Users\letie\AppData\Local\Temp\tmpd9lg8svc\1SoaebZiXaU8k10roMiogiVwUAQKRW1AI.pdf with error: RetryError[<Future at 0x193877ae5d0 state=finished raised PdfStreamError>]. Skipping...


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)


Anzahl eindeutiger Dokumente: 85


In [5]:


loader = DirectoryLoader("./bendig/", glob= "./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

Ignoring wrong pointing object 10 0 (offset 0)


In [6]:
from langchain.schema import Document

# Neue Liste mit korrekten `Document`-Objekten erstellen
corrected_documents = [
    Document(page_content=doc.text_resource.text, metadata=doc.metadata) for doc in new_documents
]




In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(corrected_documents)

In [8]:
print(texts)

[Document(metadata={'page_label': '1', 'file_name': '10_6OsnGAeDrJsFdhOSL2y3SAbNQB_Yes.pdf', 'file id': '10_6OsnGAeDrJsFdhOSL2y3SAbNQB_Yes', 'author': 'sebastian.e.becker', 'file path': 'Cases/2020-05-06_Pohler_Unilux_AufhebungsV/200605_OrgDok selbst.pdf', 'mime type': 'application/pdf', 'created at': '2025-01-14T20:45:46.254Z', 'modified at': '2020-06-05T09:00:07.000Z'}, page_content='05/06/2020 Legal Hero GmbH Mail - EL-62010 - Aufhebungsvertrag: Unterschrift & Ausfertigung\nhttps://mail.google.com/mail/b/AL_L4G8tvTi9w2EE1cAxxU-Q0wt1IxiBJGgG-zfLrRF5ms5OL8dE/u/0?ik=7cdbe24998&view=pt&search=all&permmsgid=ms… 1/2\nService Legal Hero <service@legalhero.de>\nEL-62010 - Aufhebungsvertrag: Unterschrift & Ausfertigung\nHans-Jörg Pohler <h-j.pohler@t-online.de> Fri, Jun 5, 2020 at 9:36 AM\nTo: Service Legalhero <service@legalhero.de>\nHallo Herr Becker,\nAnbei der gezeichnete Vertrag.\nGerne kann ich das Original an den RA senden.\nBitte teilen Sie mir hierzu noch die Anschrift mit.\nMit fre

Embed and store the embedding


In [9]:
#supplying a persist_directory will store embedding on disk
persist_directory = "db1"

## OpenAIEmbeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


C:\Users\letie\AppData\Local\Temp\ipykernel_6500\3519870821.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


Make a retriever

In [69]:
def generate_multi_query(query, model="gpt-3.5-turbo"):

    prompt = """
Du bist ein sachkundiger Rechtsanwaltsassistent für die Recherche.
Deine Nutzer stellen viele Anfragen zu Briefen und Gerichtsakten.
Für die gegebene Frage schlage bis zu 5 verwandte Fragen vor, um ihnen zu helfen, die benötigten Informationen zu finden.
Stelle prägnante, themenbezogene Fragen (ohne zusammengesetzte Sätze), die verschiedene Aspekte des Themas abdecken.
Stelle sicher, dass jede Frage vollständig und direkt mit der ursprünglichen Anfrage verbunden ist.
Liste jede Frage in einer eigenen Zeile ohne Nummerierung.
                """

    messages = [
        {
            "role": "system",
            "content": prompt,
        },
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content


original_query = "Was hat Bendig am Ende seines Falles bekommen?"
aug_queries = generate_multi_query(original_query)

# 3. Erweiterten Query-String für die Dokumentensuche erstellen
expanded_query = " ".join([original_query] + aug_queries)

In [84]:
retriever = vectordb.as_retriever()

corrected_documents= retriever.get_relevant_documents(expanded_query)

In [85]:
retriever = vectordb.as_retriever(search_kwargs={ "k": 4})

In [86]:
retriever.search_type

'similarity'

In [87]:
retriever.search_kwargs

{'k': 4}

In [22]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [31]:
import openai

In [32]:
client = openai.Client()  # ✅ Richtig


In [88]:
original_query = "Wann hat Bendig sein Arbeitsverältnis beendet?"
aug_queries = generate_multi_query(original_query)
print(aug_queries)

['- Wie lange war Bendig bei seinem letzten Arbeitgeber beschäftigt?', '- Gibt es Hinweise auf das genaue Datum, an dem Bendig sein Arbeitsverhältnis beendet hat?', '- Wurde das Ende des Arbeitsverhältnisses von Bendig offiziell dokumentiert?', '- Hat Bendig eine Kündigung eingereicht oder wurde er entlassen?', '- Gab es spezifische Gründe, warum Bendig sein Arbeitsverhältnis beendet hat?']


Make a chain

In [15]:
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    output_key="result"  # Hier wird festgelegt, dass nur "result" gespeichert wird
)


C:\Users\letie\AppData\Local\Temp\ipykernel_6500\2802410121.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [90]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Definiere dein individuelles Prompt-Template
custom_prompt = PromptTemplate(
    input_variables=["context", "original_query"],
    template="Beantworte die folgende Frage basierend auf dem gegebenen Kontext:\n\n"
             "Kontext:\n{context}\n\n"
             "Frage: {original_query}\n\n"
             "Antwort:"
)

# RetrievalQA mit dem benutzerdefinierten Prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever= retriever,
    return_source_documents=True,
    memory=memory,
    chain_type_kwargs={"prompt": custom_prompt,
                       "output_key": "result"}  # Hier wird das Prompt eingebunden
)


In [91]:
qa_chain = RetrievalQA.from_chain_type (llm = OpenAI(),
                                       chain_type="stuff",
                                       retriever=retriever,
                                       return_source_documents=True,
                                       memory=memory)

In [18]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nQuellen:')
    for source in llm_response["source_documents"]:
        print(source.metadata.get('file path', 'Unbekannte Quelle'))


In [ ]:
query = "Was hat Bendig bekommen am Ende seines Falles ?"
llm_response = qa_chain(expanded_query)
process_llm_response(llm_response)

 Bendig hat eine Abfindung in Höhe von 30.000,00 € Brutto erhalten. Ja, Bendig hat sein Ziel im Fall erreicht und war zufrieden mit dem Ausgang des Falles. Es gab eine außergerichtliche Einigung am Ende des Falles, jedoch wurde kein Urteil gesprochen.


Quellen:
Cases/2020-09-09_Meyer_Aldi_Kuendigung/VergleichsentwurfKKP20201029.DOC
Cases/2020-09-09_Meyer_Aldi_Kuendigung/VergleichsentwurfKKP20201029.DOC
Cases/2020-09-09_Meyer_Aldi_Kuendigung/20201102_Vergleichsentwurf_V2.0.pdf
Cases/2020-09-09_Meyer_Aldi_Kuendigung/20201102_Vergleichsentwurf_Anm LHL.pdf


: 

In [20]:

# Den Chatverlauf aus dem Memory abrufen
chat_history = memory.chat_memory.messages

# Den Verlauf formatieren und anzeigen
for message in chat_history:
    if message.type == "human":
        print(f"User: {message.content}")
    elif message.type == "ai":
        print(f"AI: {message.content}")

User: Was hat Bendig bekommen am Ende seines Falles ?
AI:  Eine ordnungsgemäß ausgefüllte Bescheinigung, ein wohlwollendes und qualifiziertes Endzeugnis mit einer Führungs- und Leistungsbeurteilung mit der Note sehr gut und eine Erwähnung im Sozialverhalten "stets vorbildlich".


In [21]:
print(qa_chain.memory.buffer)

[HumanMessage(content='Was hat Bendig bekommen am Ende seines Falles ?', additional_kwargs={}, response_metadata={}), AIMessage(content=' Eine ordnungsgemäß ausgefüllte Bescheinigung, ein wohlwollendes und qualifiziertes Endzeugnis mit einer Führungs- und Leistungsbeurteilung mit der Note sehr gut und eine Erwähnung im Sozialverhalten "stets vorbildlich".', additional_kwargs={}, response_metadata={})]
